<a href="https://colab.research.google.com/github/nobikan/PythonUrbanITMO2023/blob/final-task/final_task_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Итоговый проект. Улицы. Объекты культурного наследия.

In [ ]:
# TODO собрать установку всех необходимых модулей в одном месте
!pip install geopandas mapclassify  # mapclassify для визуализации
!pip install osmnx  # устанавливаем модуль osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 2.1 MB/s eta 0:00:00


In [ ]:
# TODO собрать импорты всех модулей в одном месте
import osmnx as ox
import geopandas as gd

In [44]:

TILES = "CartoDB positron"  # Название подложки для карт

# TODO указать любой район Санкт-Петербург из OSM https://wiki.openstreetmap.org/wiki/RU:%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3/%D0%A0%D0%B0%D0%B9%D0%BE%D0%BD%D1%8B
TERRITORY_NAME = 'Адмиралтейский'  # название территории для которой будут строиться слои

# TODO указать ссылку на файл из вашего github репозитория, которая начинается с https://raw.githubusercontent.com/
KGIOP_FILE_URL = "https://raw.githubusercontent.com/aeksei/PythonUrbanITMO2023/main/geojson_layers/bridges.geojson"  # ссылка на слой с объектами культурного наследия
STREETS_FILE_URL = "https://raw.githubusercontent.com/aeksei/PythonUrbanITMO2023/main/geojson_layers/streets.geojson"  # ссылка на слой с улицами

## Территория

### Загрузка территории из OSM (Extract)

In [45]:
# TODO загрузить geodataframe с геометрией для территории TERRITORY_NAME
district = ox.geocode_to_gdf(f"{TERRITORY_NAME} район, Санкт-Петербург")


## Улицы

### Загрузка файла с улицами (Extract)

In [46]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
gdf = gd.read_file(STREETS_FILE_URL, mask=district)
#gdf.explore(tiles="CartoDB positron")

### Обработка данных с улицами (Transform)

In [47]:
# TODO сгруппировать и объединить геометрии с одинаковыми названиями
gdf = gdf.dissolve(by="name")


### Сохранение слоя с улицами (Load)

In [48]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
gdf.rename_axis('Названия улиц',inplace=True)
gdf
# TODO для того чтобы переименовать индекс, нужно обратиться и нему и вызвать от него метод rename (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Index.rename.html)

,geometry
Названия улиц,
1-й мост Круштейна,"MULTILINESTRING ((3371922.223 8384393.995, 337..."
1-я Красноармейская улица,"MULTILINESTRING ((3373826.198 8381061.512, 337..."
10-я Красноармейская улица,"MULTILINESTRING ((3372133.931 8380334.118, 337..."
11-я Красноармейская улица,"MULTILINESTRING ((3373790.832 8380234.239, 337..."
12-я Красноармейская улица,"MULTILINESTRING ((3372708.718 8380047.963, 337..."
...,...
улица Степана Разина,"MULTILINESTRING ((3369019.100 8379612.857, 336..."
улица Труда,"LINESTRING (3372477.752 8383885.875, 3372458.3..."
улица Циолковского,"MULTILINESTRING ((3371725.332 8379587.412, 337..."


In [49]:
# TODO сохранить слой в географической проекции в формате GeoJSON
gdf.to_crs(4326).to_file('streets.geojson', driver='GeoJSON')

## Объекты культурного наследия

### Загрузка объектов культурного наследия (Extract)

In [50]:
# TODO отфильтровать улицы по маске геометрии территории полученной ранее
kgiop = gd.read_file(KGIOP_FILE_URL, mask=district)

### Обработка объектов культурного наследия (Transform)

In [51]:
# TODO добавить два столбца lon и lat, в которых будут долгота и широта
kgiop["lon"] = kgiop.to_crs(3857).geometry.centroid.to_crs(4326).x
kgiop["lat"] = kgiop.to_crs(3857).geometry.centroid.to_crs(4326).y

### Сохранение слоя с объектами культурного наследия (Load)

In [55]:
# TODO переименовать столбцы в русские названия, кроме столбца geometry
kgiop.rename(columns={"Type":"Тип","Name": "Название","District": "Район","City function": "Городская функция","lon": "Долгота","lat": "Широта"}, inplace=True)
kgiop

,Тип,Название,Район,Городская функция,geometry,Долгота,Широта
0,bridge,Дворцовый мост,Василеостровский,Достопримечательность,"MULTIPOLYGON (((30.30672 59.94210, 30.30682 59...",30.308074,59.941194
1,bridge,Благовещенский мост,Василеостровский,Достопримечательность,"MULTIPOLYGON (((30.28858 59.93619, 30.28860 59...",30.289586,59.934791
2,bridge,Старо-Калинкин мост,Адмиралтейский,Достопримечательность,"MULTIPOLYGON (((30.27888 59.91613, 30.27889 59...",30.279398,59.916411
3,bridge,Синий мост,Адмиралтейский,Достопримечательность,"MULTIPOLYGON (((30.30811 59.93155, 30.30813 59...",30.308963,59.931677
4,bridge,Горсткин мост,Адмиралтейский,Достопримечательность,"MULTIPOLYGON (((30.32367 59.92415, 30.32372 59...",30.323997,59.923980
...,...,...,...,...,...,...,...
61,bridge,Екатерингофский мост,Адмиралтейский,Достопримечательность,"MULTIPOLYGON (((30.26136 59.91266, 30.26135 59...",30.261940,59.912770
62,bridge,Сутугин мост,Адмиралтейский,Достопримечательность,"MULTIPOLYGON (((30.26889 59.90240, 30.26889 59...",30.269102,59.902408
63,bridge,Ново-Петергофский мост,Адмиралтейский,Достопримечательность,"MULTIPOLYGON (((30.29747 59.90880, 30.29745 59...",30.297711,59.908952
64,bridge,Галерный мост,Адмиралтейский,Достопримечательность,"MULTIPOLYGON (((30.26897 59.91533, 30.26941 59...",30.269278,59.915477


In [56]:
# TODO сохранить слой в географической проекции в формате GeoJSON
kgiop.to_crs(4326).to_file('bridges.geojson', driver='GeoJSON')